# Basic KF Kalman filter

In this notebook, we'll implement a basic Kalman Filter for a one-dimensional constant velocity model. We'll estimate the position and velocity of an object moving along a straight line given noisy position measurements.

The Kalman Filter is an optimal recursive data processing algorithm. It estimates the state of a dynamic system from a series of incomplete and noisy measurements.

In [2]:
import numpy as np
import matplotlib.pyplot as plt


In [3]:
#  Simulation Parameters
dt = 1.0          # Time step (seconds)
t = np.arange(0, 20 + dt, dt)  # Time vector from 0 to 20 seconds
t

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.,
       13., 14., 15., 16., 17., 18., 19., 20.])

In [5]:
# Ground Truth
# True initial position and velocity
true_position = 0.0      # meters
true_velocity = 1.0      # meters/second

# Generate true position data
true_positions = true_position + true_velocity * t
true_positions

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.,
       13., 14., 15., 16., 17., 18., 19., 20.])

In [7]:
# Noisy Measurements
# Measurement noise standard deviation
measurement_noise_std = 2.0  # meters

# Generate noisy measurements
measurements = true_positions + np.random.normal(0, measurement_noise_std, size=t.shape)
measurements


array([-1.00383474,  0.5885241 ,  0.9130795 ,  2.17506496,  6.04220753,
        5.32491572,  6.16190117,  7.23719859,  7.80397443, 10.10490933,
        9.46401587,  9.98219217, 11.40886643, 15.62755136, 15.08053681,
       13.75314478, 16.48039285, 17.97014763, 20.6962635 , 18.11247943,
       19.44683793])

## Kalman Filter Parameters

In [9]:
# State transition matrix
A = np.array([[1, dt],
              [0, 1]])

# Observation matrix
H = np.array([[1, 0]])

# Process noise covariance matrix
process_noise_std = 1e-5
Q = process_noise_std * np.array([[dt**4/4, dt**3/2],
                                  [dt**3/2, dt**2]])

# Measurement noise covariance
R = np.array([[measurement_noise_std**2]])

# Initial state estimate
x_est = np.array([[0],
                  [0]])  # Start with zero position and velocity

# Initial covariance estimate
P = np.eye(2) * 500  # Large initial uncertainty


## Storage for estimates

In [ ]:
# Lists to store estimates
estimated_positions = []
estimated_velocities = []
position_uncertainties = []
velocity_uncertainties = []


## Kalman Filter Algorithm

for idx in range(len(t)):
    # Prediction Step
    x_pred = A @ x_est
    P_pred = A @ P @ A.T + Q

    # Measurement Update Step
    z = measurements[idx]
    y = z - (H @ x_pred)  # Measurement residual
    S = H @ P_pred @ H.T + R  # Residual covariance
    K = P_pred @ H.T @ np.linalg.inv(S)  # Kalman gain

    x_est = x_pred + K @ y
    P = (np.eye(2) - K @ H) @ P_pred

    # Store estimates and uncertainties
    estimated_positions.append(x_est[0, 0])
    estimated_velocities.append(x_est[1, 0])
    position_uncertainties.append(P[0, 0])
    velocity_uncertainties.append(P[1, 1])
